In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import google, openweather

In [8]:
file = 'weather.csv'
df=pd.read_csv(file)

print(df.columns)

Index(['Unnamed: 0', 'weather', 'base', 'visibility', 'dt', 'timezone', 'id',
       'name', 'cod', 'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity',
       'main.sea_level', 'main.grnd_level', 'wind.speed', 'wind.deg',
       'wind.gust', 'clouds.all', 'sys.country', 'sys.sunrise', 'sys.sunset',
       'sys.type', 'sys.id', 'message', 'rain.1h', 'snow.1h'],
      dtype='object')


In [6]:
gmaps.configure(api_key=google)

In [9]:
locations = df[["coord.lat", "coord.lon"]].astype(float)

humidity = df["main.humidity"].astype(float)

In [15]:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
hotel_df = df.loc[(df["main.temp"] <85), :]
hotel_df = hotel_df.loc[(df["main.temp"] >75), :]
hotel_df= hotel_df.loc[(df["wind.speed"] <10), :]
hotel_df= hotel_df.loc[(df["wind.gust"] <4), :]
hotel_df= hotel_df.loc[(df["clouds.all"] <5), :]
hotel_df

,Unnamed: 0,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,...,wind.gust,clouds.all,sys.country,sys.sunrise,sys.sunset,sys.type,sys.id,message,rain.1h,snow.1h
119,0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000.0,1.620917e+09,28800.0,2062276.0,Roebourne,200,117.1333,...,2.77,0.0,AU,1.620859e+09,1.620899e+09,NaN,NaN,NaN,NaN,NaN
223,0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000.0,1.620917e+09,28800.0,1720223.0,Callaguip,200,120.4867,...,3.06,4.0,PH,1.620855e+09,1.620901e+09,NaN,NaN,NaN,NaN,NaN
304,0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000.0,1.620917e+09,7200.0,214389.0,Kaniama,200,24.1833,...,1.01,0.0,CD,1.620880e+09,1.620922e+09,NaN,NaN,NaN,NaN,NaN
446,0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000.0,1.620917e+09,28800.0,6620339.0,Karratha,200,116.8463,...,2.55,0.0,AU,1.620859e+09,1.620899e+09,NaN,NaN,NaN,NaN,NaN


In [32]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

api_pull = []

for x, y in zip(hotel_df['coord.lat'],hotel_df['coord.lon']):
    params = {
        "location": f"{x},{y}",
        "keyword": 'hotel',
        "radius": 5000,
        "type": 'hotel',
        "key": google
    }
    response = requests.get(base_url, params=params)
    api_pull.append(response.json())
    #print(x)
    #print(y)


In [41]:
hotels= []

for n in range(0,4):
    try:
        hotels.append(api_pull[n]['results'][0])
    except IndexError:
        print('No Results!')

No Results!


In [39]:
api_pull[3]

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -20.7376386, 'lng': 116.8419976},
    'viewport': {'northeast': {'lat': -20.73629782010728,
      'lng': 116.8431256298928},
     'southwest': {'lat': -20.73899747989272, 'lng': 116.8404259701073}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Karratha International Hotel',
   'opening_hours': {'open_now': False},
   'photos': [{'height': 2736,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102989211467818052053">A Google User</a>'],
     'photo_reference': 'ATtYBwJxQgkpA5sAjyodWoW7ksFIjikRamNkKmPx3vC_OFQuk9WZZkSBPgKDDs-dCN0IsvgBl1qGkzOMS4dPcjkB2WUeAmERcScocye2l9fxdpyeq0iSM2as7hXiizxowvkARP7hHZM1BjiBUANhaJckhlz_8DX8muMxiJwRUhyoAgGE41HQ',
     'width': 3648}],
   'place_id': 'ChIJqyKYq9YU9isREHRhm9HC6bA',
   'plus_code': {'compound_code': '7R6R+WQ Karratha, Western Australia, Australia',
    'glo

In [60]:
hotel_info = []
for n in hotels:
    #lat, lon, name, city, country
    lat = n['geometry']['location']['lat']
    lon = n['geometry']['location']['lng']
    name = n['name']
    thing=(n['plus_code']['compound_code'].rsplit(',', 2)[0])
    city=thing[8:]
    country =(n['plus_code']['compound_code'].rsplit(',', 1)[-1])
    tup = (lat, lon, name, city, country)
    hotel_info.append(tup)

In [61]:
hotel_info

[(-20.7544269,
  117.1457054,
  'Latitude20 Roebourne Village',
  'Roebourne',
  ' Australia'),
 (18.1135482,
  120.5269165,
  'Bellagio Hills Hotel and Restaurant',
  'Paoay',
  ' Philippines'),
 (-20.7376386,
  116.8419976,
  'Karratha International Hotel',
  'Karratha',
  ' Australia')]

In [66]:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

hotel_locations = [[n[0],n[1]] for n in hotel_info]

hotel_layer = gmaps.symbol_layer(
    hotel_locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=[f"Name: {n[2]}\nCity: {n[3]}\nCountry: {n[4]}" for n in hotel_info]
)


fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

#The Australia ones are on top of each other

fig

Figure(layout=FigureLayout(height='420px'))

In [63]:
hotel_locations

[[-20.7544269, 117.1457054],
 [18.1135482, 120.5269165],
 [-20.7376386, 116.8419976]]